## Notebook setup

In [ ]:
import os
import io
import warnings
import datetime
import numpy as np
import base64
import pandas as pd

from IPython.display import HTML

import openbb_terminal.config_terminal as cfg
from openbb_terminal.helper_classes import TerminalStyle
from openbb_terminal import OpenBBFigure, theme

warnings.filterwarnings("ignore")

In [ ]:
# Suppressing sdk logs
from openbb_terminal.core.session.current_system import set_system_variable

set_system_variable("LOGGING_SUPPRESS", True)

# Import the OpenBB SDK
from openbb_terminal.sdk import openbb, widgets

In [ ]:
from openbb_terminal.core.session.current_user import get_current_user

user = get_current_user()

cfg.theme = TerminalStyle(
    user.preferences.MPL_STYLE,
    user.preferences.PMF_STYLE,
    user.preferences.RICH_STYLE,
)

theme.apply_style("light")

In [ ]:
stylesheet = widgets.html_report_stylesheet()

## Select Ticker

In [ ]:
# Parameters that will be replaced when calling this notebook
# Do not leave parameters blank as notebook will not run otherwise
symbol = "EURUSD"
report_name = "Forex Report for EURUSD"

In [ ]:
ticker = symbol

In [ ]:
author = ""
report_title = f"INVESTMENT RESEARCH REPORT ON {ticker.upper()}"
report_date = datetime.datetime.now().strftime("%d %B, %Y")
report_time = datetime.datetime.now().strftime("%H:%M")
report_timezone = datetime.datetime.now(datetime.timezone.utc).astimezone().tzinfo
report_title, report_date, report_time, report_timezone

In [ ]:
from_symbol = ticker[:3]
to_symbol = ticker[3:]
last_year = datetime.datetime.now() - datetime.timedelta(days=365)

ticker_data = openbb.forex.load(
    from_symbol=from_symbol,
    to_symbol=to_symbol,
    start_date=last_year.strftime("%Y-%m-%d"),
    interval="1day",
    resolution="d",
)

# 1 year historical data
ticker_data

In [ ]:
last_year = datetime.datetime.now() - datetime.timedelta(days=365)

ticker_data_interval_1month = openbb.forex.load(
    from_symbol=from_symbol,
    to_symbol=to_symbol,
    start_date=last_year.strftime("%Y-%m-%d"),
    interval="1month",
    resolution="d",
)
ticker_data_interval_1month = ticker_data_interval_1month.drop(columns="Volume")
ticker_data_interval_1month

## Data

In [ ]:
# candle

candle = openbb.forex.candle(
    data=ticker_data,
    to_symbol=to_symbol,
    from_symbol=from_symbol,
    external_axes=True,
).to_html()

In [ ]:
# candle graph with moving averages 7/14 and 30/60
candle_ma_7_14 = openbb.forex.candle(
    data=ticker_data,
    to_symbol=to_symbol,
    from_symbol=from_symbol,
    ma=[7, 14],
    external_axes=True,
).to_html()

candle_ma_30_60 = openbb.forex.candle(
    data=ticker_data,
    to_symbol=to_symbol,
    from_symbol=from_symbol,
    ma=[30, 60],
    external_axes=True,
).to_html()

In [ ]:
fwd = openbb.forex.fwd(
    to_symbol=to_symbol,
    from_symbol=from_symbol,
)

# forward rates
fwd

# Technical analysis

In [ ]:
# simple moving average
ticker_data.index.names = ["date"]

sma = openbb.ta.ma_chart(
    data=ticker_data["Close"],
    ma_type="SMA",
    symbol=ticker,
    external_axes=True,
).to_html()

In [ ]:
# exponential moving average
ema = openbb.ta.ma_chart(
    data=ticker_data["Close"],
    ma_type="EMA",
    symbol=ticker,
    external_axes=True,
).to_html()

In [ ]:
# zero lag exponential moving average

zlma = openbb.ta.ma_chart(
    data=ticker_data["Close"],
    ma_type="ZLMA",
    symbol=ticker,
    external_axes=True,
).to_html()

In [ ]:
# commodity channel index

cci = openbb.ta.cci_chart(
    data=ticker_data,
    symbol=ticker,
    external_axes=True,
).to_html()

In [ ]:
# moving average convergence/divergence

macd = openbb.ta.macd_chart(
    data=ticker_data["Adj Close"],
    symbol=ticker,
    external_axes=True,
).to_html()

In [ ]:
# fisher transform

fisher = openbb.ta.fisher_chart(
    data=ticker_data,
    symbol=ticker,
    external_axes=True,
).to_html()

In [ ]:
# aroon indicator

aroon = openbb.ta.aroon_chart(
    data=ticker_data,
    symbol=ticker,
    external_axes=True,
).to_html()

In [ ]:
# bollinger bands

bbands = openbb.ta.bbands_chart(
    data=ticker_data,
    symbol=ticker,
    external_axes=True,
).to_html()

In [ ]:
# fibonacci retracement

fib = openbb.ta.fib_chart(
    data=ticker_data,
    symbol=ticker,
    external_axes=True,
).to_html()

# Quantitative Analysis

In [ ]:
# summary

summary = openbb.qa.summary(data=ticker_data)
summary

In [ ]:
# normality statistics and tests

normality = openbb.qa.normality(
    data=ticker_data["Close"],
)
normality

In [ ]:
# rolling mean and std deviation of prices

rolling = (
    openbb.qa.rolling_chart(
        symbol=ticker,
        data=ticker_data,
        target="Close",
        external_axes=True,
    )
    .update_layout(
        legend=dict(
            x=0.01,
            y=0.99,
            xanchor="left",
            yanchor="top",
            xref="paper",
            yref="paper",
            bgcolor="rgba(0, 0, 0, 0)",
        ),
        legend2=dict(bgcolor="rgba(0, 0, 0, 0)"),
    )
    .to_html()
)

In [ ]:
# rolling kurtosis of distribution of prices

kurtosis = openbb.qa.kurtosis_chart(
    symbol=ticker,
    data=ticker_data,
    target="Close",
    external_axes=True,
).to_html()

In [ ]:
# latest news
try:
    news = openbb.news(term=ticker).head(10)
    news.columns = news.columns.str.title()

    news = news.rename(
        columns={"Date": "Published", "Description": "Title", "Url": "Link"}
    )
    news["Title"] = news.apply(
        lambda x: f'<a href="{x["Link"]}">{x["Title"]}</a>', axis=1
    )
    news = news[["Title", "Published"]]
    news["Published"] = pd.to_datetime(news["Published"]).dt.strftime("%Y-%m-%d %H:%M")

    news = news.set_index("Published")
    news.sort_index()
except:
    news = pd.DataFrame()

## Render the report template to a file

In [ ]:
body = ""


body += widgets.header(
    author,
    report_date,
    report_time,
    report_timezone,
    f"<b>INVESTMENT RESEARCH REPORT:</b> {ticker}",
    plotly_js=True,
)

body += widgets.tablinks(
    [
        "SUMMARY",
        "Overview",
        "Technical Analysis",
        "Quantitative Analysis",
    ]
)

comment = "Every analysis displayed considered every day data YTD"


# TODO: add KPIs

htmlcode = ""

# htmlcode += widgets.h(3, "KPIs")
# htmlcode += widgets.kpi(
#     [30, 70],
#     ["RSI level is oversold", "RSI level is normal", "RSI level is overbought"],
#     999,
# )
# htmlcode += widgets.kpi(
#     [0],
#     [
#         "The sum of last 10 insider trading (in millions) was negative",
#         "The sum of last 10 insider trading (in millions) was positive",
#     ],
#     999,
# )
# htmlcode += widgets.kpi(
#     [-0.1, 0.1],
#     [
#         "Last FinBrain sentiment is bearish",
#         "Last FinBrain sentiment is neutral",
#         "Last FinBrain sentiment is bullish",
#     ],
#     999,
# )

# Summary tab
htmlcode += widgets.row([widgets.h(3, "Latest news") + news.to_html(escape=False)])
htmlcode += widgets.row(
    [
        widgets.h(3, "One year historical data for 1st day each month")
        + ticker_data_interval_1month.to_html()
    ]
)
body += widgets.add_tab("SUMMARY", htmlcode)

# Overview tab
htmlcode = ""
htmlcode = widgets.row([widgets.h(3, "Candlestick chart")])
htmlcode += widgets.row([candle])
htmlcode += widgets.row(
    [widgets.h(3, "Candlestick chart with moving averages 7 and 14")]
)
htmlcode += widgets.row([candle_ma_7_14])
htmlcode += widgets.row(
    [widgets.h(3, "Candlestick chart with moving averages 30 and 60")]
)
htmlcode += widgets.row([candle_ma_30_60])
htmlcode += widgets.row([widgets.h(3, "Forward rates")])
htmlcode += widgets.row([fwd.to_html()])
body += widgets.add_tab("Overview", htmlcode, True, comment)

# Technical Analysis tab
htmlcode = widgets.row([widgets.h(3, "Simple moving average")])
htmlcode += widgets.row([sma])
htmlcode += widgets.row([widgets.h(3, "Exponential moving average")])
htmlcode += widgets.row([ema])
htmlcode += widgets.row([widgets.h(3, "Zero lag exponential moving average")])
htmlcode += widgets.row([zlma])
htmlcode += widgets.row([widgets.h(3, "Commodity channel index")])
htmlcode += widgets.row([cci])
htmlcode += widgets.row([widgets.h(3, "Moving average convergence/divergence")])
htmlcode += widgets.row([macd])
htmlcode += widgets.row([widgets.h(3, "Fisher transform")])
htmlcode += widgets.row([fisher])
htmlcode += widgets.row([widgets.h(3, "Aroon indicator")])
htmlcode += widgets.row([aroon])
htmlcode += widgets.row([widgets.h(3, "Bollinger bands")])
htmlcode += widgets.row([bbands])
htmlcode += widgets.row([widgets.h(3, "Fibonacci retracement")])
htmlcode += widgets.row([fib])
body += widgets.add_tab("Technical Analysis", htmlcode, True, comment)

# Quantitative Analysis tab
htmlcode = widgets.row([widgets.h(3, "Summary") + summary.to_html()])
htmlcode += widgets.row(
    [widgets.h(3, "Normality statistics and tests") + normality.to_html()]
)
htmlcode += widgets.row([widgets.h(3, "Rolling mean and std deviation of prices")])
htmlcode += widgets.row([rolling])
htmlcode += widgets.row([widgets.h(3, "Rolling kurtosis of distribution of prices")])
htmlcode += widgets.row([kurtosis])
body += widgets.add_tab("Quantitative Analysis", htmlcode, True, comment)

body += widgets.tab_clickable_and_save_evt()
report = widgets.html_report(title=report_name, stylesheet=stylesheet, body=body)

# # to save the results
with open(report_name + ".html", "w", encoding="utf-8") as fh:
    fh.write(report)